### Energy Consumption by Source dataset overview

Before narrowing the analysis to renewable energy, it is important to first review Europe’s overall energy mix to understand how renewables fit within it.

#### Methodology
Because the dataset contains only non-negative values and the goal is to illustrate total primary energy consumption while highlighting the contributions of individual energy sources, a stacked area chart is the most appropriate visualization.

<a href="https://www.repsol.com/en/energy-move-forward/energy/primary-energy/index.cshtml"> Primary energy </a> refers to the energy available as resources – such as the fuels burnt in power plants – before it has been transformed. This relates to the coal before it has been burned, the uranium, or the barrels of oil. Primary energy includes energy that the end user needs, in the form of electricity, transport and heating, plus inefficiencies and energy that is lost when raw resources are transformed into a usable form. _(Definition based on: Repsol – Primary Energy)_

<a href="https://ourworldindata.org/energy-substitution-method"> Substitution method </a> is one method used to adjust primary energy consumption for efficiency losses experienced by fossil fuels. It tries to adjust non-fossil energy sources to the inputs that would be needed if it was generated from fossil fuels. It assumes that wind and solar electricity is as inefficient as coal or gas. To do this, energy generation from non-fossil sources are divided by a standard ‘thermal efficiency factor’ – typically around 0.4. Nuclear power is also adjusted despite it also experiencing thermal losses in a power plant. Since it’s reported in terms of electricity output, we need to do this adjustment to calculate its equivalent input value. _(Method reference: Our World in Data – Energy Substitution Method)_

Each layer of the stacked area chart represents a different energy source, revealing how the European energy mix has evolved over time. The visualization was created using `matplotlib.stackplot()`.

To ensure accuracy and robustness in data handling throughout the analysis, unit tests were implemented for the data-processing functions (both for this visualization and others). This served both analytical reliability and as practice in incorporating basic testing workflows into the code.

_Libraries for data loading and inspection_

In [1]:
import pandas as pd
import plotly.express as px
%matplotlib inline


In [2]:
share_energy = pd.read_csv('../data/energy_consumption_by_source_europe.csv')
share_energy.head()

,Entity,Code,Year,Other renewables (including geothermal and biomass) - TWh,Biofuels consumption - TWh,Solar consumption - TWh,Wind consumption - TWh,Hydro consumption - TWh,Nuclear consumption - TWh,Gas consumption - TWh,...,Oil consumption - TWh,Other renewables (including geothermal and biomass) - TWh.1,Biofuels consumption - TWh.1,Solar consumption - TWh.1,Wind consumption - TWh.1,Hydro consumption - TWh.1,Nuclear consumption - TWh.1,Gas consumption - TWh.1,Coal consumption - TWh.1,Oil consumption - TWh.1
0,Europe,unknown,1965,9.090626,0.0,0.0,0.0,1059.5842,59.826576,1595.9036,...,6933.2324,9.090626,0.0,0.0,0.0,1059.5842,59.826576,1595.9036,9548.794,6933.2324
1,Europe,unknown,1966,12.434375,0.0,0.0,0.0,1150.1521,77.374600,1786.6346,...,7552.9653,12.434375,0.0,0.0,0.0,1150.1521,77.374600,1786.6346,9420.777,7552.9653
2,Europe,unknown,1967,13.487500,0.0,0.0,0.0,1140.6998,89.244150,2008.7863,...,8203.0420,13.487500,0.0,0.0,0.0,1140.6998,89.244150,2008.7863,9291.548,8203.0420
3,Europe,unknown,1968,15.056251,0.0,0.0,0.0,1214.1719,102.625984,2280.0083,...,8936.2490,15.056251,0.0,0.0,0.0,1214.1719,102.625984,2280.0083,9340.795,8936.2490
4,Europe,unknown,1969,16.390326,0.0,0.0,0.0,1229.7555,124.265960,2583.9653,...,9826.8690,16.390326,0.0,0.0,0.0,1229.7555,124.265960,2583.9653,9481.040,9826.8690


The chart displays Europe’s primary energy consumption by source, measured using the substitution method. The dataset comes from Our World in Data (“Energy consumption by source, Europe”) and is based on the Energy Institute’s Statistical Review of World Energy (2025). Data was unavailable for Andorra, Liechtenstein, Monaco, San Marino, and Vatican City.

Below is a summary of the dataset’s key columns:
* **Entity**: the name of the country.
* **Code**: country code.
* **Year**: observation year.
* **Other renewables (including geothermal and biomass) - TWh**: energy consumption from geothermal, biomass, waste, wave, and tidal source in Terawatt-hours.
* **Biofuels consumption - TWh**: Biofuel consumption in Terawatt-hours.
* **Solar consumption - TWh**: Solar consumption in Terawatt-hours.
* **Wind consumption - TWh**: Wind consumption in Terawatt-hours.
* **Hydro consumption - TWh**: Hydro consumption in Terawatt-hours.
* **Nuclear consumption - TWh**: Nuclear consumption in Terawatt-hours.
* **Gas consumption - TWh**: Gas consumption in Terawatt-hours.
* **Coal consumption - TWh**: Coal consumption in Terawatt-hours
* **Oil consumption - TWh**: Oil consumption in Terawatt-hours.

In [3]:
energy_cols = [
    "Other renewables (including geothermal and biomass) - TWh",
    "Biofuels consumption - TWh",
    "Solar consumption - TWh",
    "Wind consumption - TWh",
    "Hydro consumption - TWh",
    "Nuclear consumption - TWh",
    "Gas consumption - TWh",
    "Coal consumption - TWh",
    "Oil consumption - TWh"
]
share_energy[energy_cols] = share_energy[energy_cols].apply(pd.to_numeric, errors='coerce')

df_yearly = share_energy.groupby("Year")[energy_cols].sum().reset_index()

df_yearly['Total_energy'] = df_yearly[energy_cols].sum(axis=1)

df_melted = df_yearly.melt(
    id_vars=['Year', 'Total_energy'],
    value_vars=energy_cols,
    var_name='Source',
    value_name='Energy_TWh'
)

source_name_map = {
    "Other renewables (including geothermal and biomass) - TWh": "Other renewables",
    "Biofuels consumption - TWh": "Biofuels",
    "Solar consumption - TWh": "Solar",
    "Wind consumption - TWh": "Wind",
    "Hydro consumption - TWh": "Hydropower",
    "Nuclear consumption - TWh": "Nuclear",
    "Gas consumption - TWh": "Gas",
    "Coal consumption - TWh": "Coal",
    "Oil consumption - TWh": "Oil"
}
df_melted['Source'] = df_melted['Source'].replace(source_name_map)

hover_text_dict = {}
for _, row in df_yearly.iterrows():
    hover_text = f"<b>Year: {int(row['Year'])}</b><br>"
    for col in energy_cols:
        display_name = source_name_map[col]
        hover_text += f"{display_name}: {row[col]:,.2f} TWh<br>"
    hover_text += f"<b>Total: {row['Total_energy']:,.2f} TWh</b>"
    hover_text_dict[row['Year']] = hover_text

df_melted['hover_text'] = df_melted['Year'].map(hover_text_dict)

fig = px.area(
    df_melted,
    x='Year',
    y='Energy_TWh',
    color='Source',
    category_orders={'Source': list(source_name_map.values())[::-1]},
    hover_data={'Energy_TWh': False, 'Total_energy': False},
    title='Europe: Energy Consumption by Source (TWh)'
)

fig.update_traces(
    text=df_melted['hover_text'],
    hovertemplate="%{text}<extra></extra>"
)

fig.update_layout(
    yaxis_title='Energy Consumption (TWh)',
    xaxis_title='Year',
    template='plotly_white',
    legend_traceorder="normal",
    legend=dict(traceorder='reversed')
)

fig.show()

#### Key Turning Points and Trends Across Six Decades:
* **Between 1965 and 1975: Fossil Fuel Dominance.** Europe’s energy system was overwhelmingly dependent on fossil fuels. Oil consumption grew markedly—from 36.10% in 1965 to 44.41% in 1975 (a 23% increase)—reflecting rapidly expanding global demand prior to the mid-1970s energy crises.
* **Between 1975 and 1990: Diversification After the Oil Crisis.** Following the 1973 OPEC oil embargo, Europe intensified efforts to diversify its energy portfolio. Nuclear power expanded rapidly due to technological advances and new reactor construction. Wind first appeared in the dataset in 1978, followed by solar in 1989, marking the earliest steps toward modern renewable energy adoption.
* **Between 1990 and 2005: Renewables Emerge.** Rising environmental awareness, higher fossil-fuel prices, and early climate policy frameworks helped bring renewable energy into the mainstream. Wind power, biofuels, geothermal, and other renewables grew steadily, representing the first meaningful expansion of low-carbon energy sources.
* **Between 2005 and 2016: Rapid Growth Driven by Costs and Policy.** Global investment surged and the costs of wind and solar technologies fell sharply. EU-level subsidies and renewable energy directives accelerated deployment. By 2016, renewables were increasingly displacing fossil fuels across the European energy mix, with wind and solar achieving record levels of expansion.
* **Between 2015 and 2024: Green Deal Momentum.** Under the influence of the European Green Deal, coal consumption declined sharply. Renewable energy rose from 12.42% in 2015 to 17.85% in 2024—an increase of 43.7% in relative share.

#### Additional Observations:
* The graph shows two major declines in total energy consumption: in 2009 after the global financial crisis and in 2020 due to the COVID-19 pandemic. In both cases, fossil fuel use dropped sharply while renewables continued to rise.
* A general decline in electricity consumption has been visible since the early 1990s, though this trend appears to be slowing down.
* While renewables continue to grow, nuclear power has been gradually declining since 2004.
* Hydropower remains the largest modern renewable source, but both wind and solar have seen rapid and sustained growth.

#### Future of Renewable energy industry:
Based on the graph and supplementary readings, the analysis suggests that although renewable energy technologies have advanced significantly, the pace of transition remains too slow to rapidly replace fossil fuels, guarantee energy security, and deliver the emissions reductions required to meet climate goals. Achieving deep decarbonization will require shifting energy systems decisively toward low-carbon sources.

For example, according to Eurostat, renewables accounted for 24.5% of the EU’s gross final energy consumption in 2023, still far below the 42.5% target for 2030. This means the EU would need to nearly double the recent rate of renewable deployment to remain on target.

While the situation might seem discouraging at first glance, rapid improvement remains possible with accelerated policy action, including increased subsidies and stronger investment frameworks. There are positive examples: Lithuania, Poland, the Netherlands, and Denmark increased their renewable energy shares by 2 percentage points within 2024, showing that swift progress can occur within short timeframes.


#### Sources:
* _Data source: Our World in Data - Energy Mix, by Hannah Richie and Pablo Rosado (2020):_ <a href="https://ourworldindata.org/energy-mix"> link </a>
* _Office of The Historian - Oil Embargo, 1973-1974:_ <a href="https://history.state.gov/milestones/1969-1976/oil-embargo"> link </a>
* _Wikipedia - History of nuclear power:_ <a href="https://en.wikipedia.org/wiki/History_of_nuclear_power"> link </a>
* _European Environment Agency - Renewables accounted for vast majority of new EU power capacity in 2016:_ <a href="https://www.eea.europa.eu/highlights/renewables-accounted-for-vast-majority"> link </a>
* _Eurostat - Energy Statistics – An Overview:_ <a href="https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Energy_statistics_-_an_overview"> link </a>
* _Eurostat - Renewable energy supply grew by 3.4% in 2024:_ <a href="https://ec.europa.eu/eurostat/web/products-eurostat-news/w/ddn-20250702-1"> link </a>
* _European Environment Agency - Share of energy consumption from renewable sources in Europe:_ <a href="https://www.eea.europa.eu/en/analysis/indicators/share-of-energy-consumption-from?utm_source=chatgpt.com"> link </a>

